In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
#Import libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from sklearn.model_selection import train_test_split

In [ ]:
#Store Data

ratings_dataset = pd.read_csv('https://storage.googleapis.com/bogorism/dataPlaces/Rating.csv', encoding='unicode_escape')
tourism_dataset = pd.read_csv('https://storage.googleapis.com/bogorism/dataPlaces/Place.csv', encoding='unicode_escape')

ratings_dataset.head()

,User_Id,Place_Id,Place_Ratings
0,1,5,5
1,1,101,4
2,1,20,4
3,1,103,3
4,1,89,3


In [ ]:
total_place = ratings_dataset.Place_Id.nunique()
total_user = ratings_dataset.User_Id.nunique()

In [ ]:
train, test = train_test_split(ratings_dataset, test_size=0.1)
FEATURES = 4

In [ ]:
print(train.shape)
print(test.shape)

(2491, 3)
(277, 3)


In [ ]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1)
])

In [ ]:
EMBEDDING_DIM = 3

#input layers
place_input = Input(shape = 1)
user_input = Input(shape = 1)

#embedding layers
place_embedding = Embedding(total_place+1, EMBEDDING_DIM)(place_input)
user_embedding = Embedding(total_user+1, EMBEDDING_DIM)(user_input)

#flatten layers
place_flat = Flatten()(place_embedding)
user_flat = Flatten()(user_embedding)

#output layers
output = Dot(1)([place_flat, user_flat])

model = Model([place_input, user_input], [output])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 3)         363         ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 3)         903         ['input_2[0][0]']                
                                                                                              

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
x = train.Place_Id, train.User_Id
y= train.Place_Ratings

In [ ]:
model.fit(x, y, epochs = 30, batch_size = 5)

Epoch 1/30
499/499 [==============================] - 5s 4ms/step - loss: 11.0816
Epoch 2/30
499/499 [==============================] - 2s 3ms/step - loss: 11.0546
Epoch 3/30
499/499 [==============================] - 2s 3ms/step - loss: 10.9105
Epoch 4/30
499/499 [==============================] - 2s 5ms/step - loss: 10.5609
Epoch 5/30
499/499 [==============================] - 3s 5ms/step - loss: 9.9965
Epoch 6/30
499/499 [==============================] - 3s 5ms/step - loss: 9.2395
Epoch 7/30
499/499 [==============================] - 3s 6ms/step - loss: 8.3360
Epoch 8/30
499/499 [==============================] - 3s 6ms/step - loss: 7.3475
Epoch 9/30
499/499 [==============================] - 3s 6ms/step - loss: 6.3481
Epoch 10/30
499/499 [==============================] - 2s 5ms/step - loss: 5.3919
Epoch 11/30
499/499 [==============================] - 2s 3ms/step - loss: 4.5334
Epoch 12/30
499/499 [==============================] - 2s 3ms/step - loss: 3.8045
Epoch 13/30
499/499 [

In [ ]:
model.evaluate(x = [test.Place_Id, test.User_Id], y= [test.Place_Ratings])

9/9 [==============================] - 0s 2ms/step - loss: 2.3100


2.310004472732544

In [ ]:
model.save('model.h5')

In [ ]:
def place_recommendations(User_Id,tourism_dataset, model):
  tourism_dataset = tourism_dataset.copy()
  User_ids = np.array([User_Id] * len(tourism_dataset))
  results = model([tourism_dataset.Place_Id.values, User_ids]).numpy().reshape(-1)

  tourism_dataset['predicted_rating'] = pd.Places(results)
  tourism_dataset = tourism_dataset.sort_values('predicted_rating', ascending = False)

  print(f'Recommendations for user {User_Id}')
  return tourism_dataset

  get_recommendations(1, tourism_dataset, model)

In [ ]:
def get_embedding(tourism_dataset, model):
  vectors = model.layers[2](tourism_dataset.Place_Id.values).numpy()
  vectors = pd.DataFrame(vectors)
  vectors.to_csv('vectors.tsv', sep='\t', header=False)

  tourism_dataset.to_csv('meta.tsv', sep='\t')

  get_embedding(tourism_dataset, model)